# Tracks

Generate tracks dataframe for analysis in R: `tracks.csv`

In [ ]:
from climada.hazard import TCTracks
import numpy as np
import pandas as pd
from pathos.pools import ProcessPool
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# set parameters
start_year = 1980
end_year = 2023

pool = ProcessPool(nodes=8)
tracks = TCTracks.from_ibtracs_netcdf(year_range=(start_year, end_year))

n_TCs = tracks.size

time_start = [track.indexes['time'][0] for track in tracks.data]
time_end = [track.indexes['time'][-1] for track in tracks.data]

times_dt = [time - pd.Timestamp("1/1/1980") for time in time_start]
times_dt_days = [time.total_seconds() / (60*60*24) for time in times_dt]
times_dt_years = [time.total_seconds() / (60*60*24*365.25) for time in times_dt]

max_sw = [np.max(track['max_sustained_wind'].values) for track in tracks.data]
min_sw = [np.min(track['max_sustained_wind'].values) for track in tracks.data]
mean_sw = [np.mean(track['max_sustained_wind'].values) for track in tracks.data]

lifetimes = [(time_end[i] - time_start[i]).total_seconds()/(60*60*24*365.25) for i in range(n_TCs)]

tracks_df = pd.DataFrame({
    'basin': [track['basin'].values[0] for track in tracks.data],
    'TC_num': range(1,n_TCs+1),	
    'days_since_1980': times_dt_days,
    'years_since_1980': times_dt_years,
    'year': [track.indexes['time'].year[0] for track in tracks.data],
    'month': [track.indexes['time'].month[0] for track in tracks.data],
    'max_sw': max_sw,
    'min_sw': min_sw,
    'mean_sw': mean_sw,
    'category': [track.category for track in tracks.data],
    'lat': [track.lat.values[0] for track in tracks.data],
    'lon': [track.lon[0].values % 360 for track in tracks.data],
    'lifetime': lifetimes
})
tracks_df.index = time_start
tracks_df.sort_index(inplace=True)

In [ ]:
# Save dataframe
tracks_df.to_csv('Data/tracks.csv')